In [3]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

/Users/paulag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [5]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [6]:
oni = ONI(nino34)

In [7]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [8]:
clm = climatology(nino34)

In [9]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [10]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [24]:
T = 7      # prediction timeline
H = 48   # history used for prediction
include_month = 1           # 1 if we use the month as a feature, 0 otherwise
n_classes = 13               # number of classes (El Nino, El Nina, No Event)
threshold = 0.5         
signal = np.array(nino34[12 * 50:])   # data used for training/testing
length = signal.shape[0]    # number of data points

In [29]:
np.random.seed(0)

size = length - H - T       # effective dataset size
        
# create the 'history matrix'
data = np.ndarray((size, H + include_month))
for i in range(size):
    if(include_month == False):
         data[i] = signal[i:(i + H)]
    else:
        data[i] = np.append(signal[i:(i + H)], (i + H + T) % 12)

labels = np.ndarray((size), dtype = int)
        
for i in range(size):
    labels[i] = oanm[i + H + T] // threshold + 6

split = size // 10      
shuffle = np.random.permutation(size)
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[8 * split: 9 * split])
test_ind = np.array(shuffle[9 * split: size])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])
        
mean = np.mean(train, axis = 0)
std = np.std(train, axis = 0)

train_n = np.divide(
        train - np.outer(np.ones(train.shape[0]), mean),
        np.outer(np.ones(train.shape[0]), std))

val_n = np.divide(
        val - np.outer(np.ones(val.shape[0]), mean),
        np.outer(np.ones(val.shape[0]), std))

test_n = np.divide(
        test - np.outer(np.ones(test.shape[0]), mean),
        np.outer(np.ones(test.shape[0]), std))
        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    encoded_train_labels = tf.one_hot(train_labels, depth = n_classes).eval()
    encoded_val_labels = tf.one_hot(val_labels, depth = n_classes).eval()
    encoded_test_labels = tf.one_hot(test_labels, depth = n_classes).eval()

In [33]:
(d, N1, N2, N3) = (0.3, 768, 768, 384)
epochs = 25
        
acc = 0
loss = 0
        
model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N1, activation=tf.nn.relu),
              tf.keras.layers.Dropout(d),
              tf.keras.layers.Dense(N2, activation=tf.nn.relu),
              tf.keras.layers.Dropout(d),
              tf.keras.layers.Dense(N3, activation=tf.nn.relu),
              tf.keras.layers.Dropout(d),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])

model.reset_states()

model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.fit(train_n, encoded_train_labels, epochs = epochs)
(loss, acc) = model.evaluate(val_n, encoded_val_labels, verbose = 0)

Epoch 1/25
5712/5712 [==============================] - 4s 630us/step - loss: 1.9294 - acc: 0.2433
Epoch 2/25
5712/5712 [==============================] - 3s 531us/step - loss: 1.8823 - acc: 0.2567
Epoch 3/25
5712/5712 [==============================] - 3s 480us/step - loss: 1.8572 - acc: 0.2698
Epoch 4/25
5712/5712 [==============================] - 3s 484us/step - loss: 1.8236 - acc: 0.2806
Epoch 5/25
5712/5712 [==============================] - 2s 434us/step - loss: 1.7759 - acc: 0.3029
Epoch 6/25
5712/5712 [==============================] - 2s 426us/step - loss: 1.7174 - acc: 0.3309
Epoch 7/25
5712/5712 [==============================] - 3s 446us/step - loss: 1.6490 - acc: 0.3564
Epoch 8/25
5712/5712 [==============================] - 2s 426us/step - loss: 1.5596 - acc: 0.3857
Epoch 9/25
5712/5712 [==============================] - 3s 457us/step - loss: 1.4538 - acc: 0.4266
Epoch 10/25
5712/5712 [==============================] - 3s 451us/step - loss: 1.3393 - acc: 0.4774
Epoch 11/

NameError: name 't' is not defined

In [34]:
print(acc)

0.5238095239764836


In [51]:
pred = model.predict_classes(val_n, verbose = 0)
dist = np.abs(pred - val_labels)
MSE = np.mean(dist * dist)
easy_acc = np.mean(dist < 6)
easy_acc2 = np.mean(dist < 3)
print(MSE)
print(easy_acc)
print(easy_acc2)

2.033613445378151
0.9957983193277311
0.8991596638655462


In [52]:
from sklearn import multiclass
from sklearn.svm import SVC

clf = multiclass.OneVsRestClassifier(SVC(kernel='rbf', random_state = 0))
clf.fit(train, train_labels)
acc_SVC = clf.score(val, val_labels)

print(np.around(acc_SVC * 100, decimals = 2), "%")

34.45 %
